In [5]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import math
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(RANDOM_SEED)

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

# Define target variables and features
TARGET_VARIABLES = [
    'Aroma', 'Flavor', 'Aftertaste', 'Acidity', 'Body', 'Balance', 
    'Uniformity', 'Clean Cup', 'Sweetness', 'Cupper Points', 
    'Total Cup Points', 'Moisture', 'Category One Defects', 
    'Quakers', 'Category Two Defects', 'Total Yield'
]

# Load and preprocess data
def load_data(file_path='final_coffee.csv'):
    print("Loading data...")
    df = pd.read_csv(file_path)
    
    # Get all columns except target variables
    feature_columns = [col for col in df.columns if col not in TARGET_VARIABLES]
    
    # Split features and targets
    X = df[feature_columns]
    y = df[TARGET_VARIABLES]
    
    # Handle categorical features - one-hot encode
    X_encoded = pd.get_dummies(X, drop_first=False)
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X_encoded, y, test_size=0.2, random_state=RANDOM_SEED
    )
    
    # Scale features
    feature_scaler = StandardScaler()
    X_train_scaled = feature_scaler.fit_transform(X_train)
    X_test_scaled = feature_scaler.transform(X_test)
    
    # Scale targets
    target_scalers = {}
    y_train_scaled = np.zeros_like(y_train.values, dtype=np.float32)
    y_test_scaled = np.zeros_like(y_test.values, dtype=np.float32)
    
    print("Target value ranges:")
    for i, col in enumerate(TARGET_VARIABLES):
        print(f"{col}: min={y_train[col].min():.2f}, max={y_train[col].max():.2f}, mean={y_train[col].mean():.2f}, std={y_train[col].std():.2f}")
        
        scaler = StandardScaler()
        y_train_scaled[:, i] = scaler.fit_transform(y_train[col].values.reshape(-1, 1)).flatten()
        y_test_scaled[:, i] = scaler.transform(y_test[col].values.reshape(-1, 1)).flatten()
        target_scalers[col] = scaler
    
    # Convert to tensors
    X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
    X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train_scaled, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test_scaled, dtype=torch.float32)
    
    # Print input dimension
    print(f"Input dimension: {X_train_tensor.shape[1]}")
    
    return (X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor, 
            X_train, X_test, y_train, y_test, feature_scaler, target_scalers)

# Simple yet effective TabNet implementation
class TabNet(nn.Module):
    def __init__(self, input_dim, output_dim=1, feature_dim=64, output_dim_step=8, n_steps=3, gamma=1.3, n_shared=2, epsilon=1e-15):
        super(TabNet, self).__init__()
        
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.feature_dim = feature_dim
        self.output_dim_step = output_dim_step
        self.n_steps = n_steps
        self.gamma = gamma
        self.epsilon = epsilon
        
        # Feature encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 2*feature_dim),
            nn.BatchNorm1d(2*feature_dim)
        )
        
        # Shared feature transformers for feature selection
        self.shared_feature_transform = nn.ModuleList()
        for i in range(n_shared):
            if i == 0:
                self.shared_feature_transform.append(nn.Sequential(
                    nn.Linear(feature_dim, feature_dim),
                    nn.BatchNorm1d(feature_dim),
                    nn.ReLU()
                ))
            else:
                self.shared_feature_transform.append(nn.Sequential(
                    nn.Linear(feature_dim, feature_dim),
                    nn.BatchNorm1d(feature_dim),
                    nn.ReLU()
                ))
        
        # Feature transformer for each step
        self.feature_transforms = nn.ModuleList()
        for i in range(n_steps):
            transform = nn.Sequential(
                nn.Linear(feature_dim, feature_dim),
                nn.BatchNorm1d(feature_dim),
                nn.ReLU(),
                nn.Linear(feature_dim, output_dim_step),
                nn.BatchNorm1d(output_dim_step)
            )
            self.feature_transforms.append(transform)
        
        # Feature selection masks
        self.attentions = nn.ModuleList()
        for i in range(n_steps):
            attention = nn.Sequential(
                nn.Linear(feature_dim, input_dim),
                nn.BatchNorm1d(input_dim)
            )
            self.attentions.append(attention)
        
        # Final output layer
        self.final_layer = nn.Linear(output_dim_step * n_steps, output_dim)
        
        # Initialize weights
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.kaiming_normal_(module.weight, mode='fan_in', nonlinearity='relu')
            if module.bias is not None:
                nn.init.zeros_(module.bias)
        elif isinstance(module, nn.BatchNorm1d):
            nn.init.ones_(module.weight)
            nn.init.zeros_(module.bias)
    
    def forward(self, x):
        # Initial encoding
        x_encoded = self.encoder(x)
        
        # Split to create initial feature tensor
        feature = torch.split(x_encoded, self.feature_dim, dim=1)[0]
        
        # Used to compute the mask
        prior = torch.ones(x.shape).to(x.device)
        
        # Storage for step outputs
        step_output = []
        
        # For each step, apply feature selection and transformation
        for step_idx in range(self.n_steps):
            # Transform features through shared layers
            for layer in self.shared_feature_transform:
                feature = layer(feature)
            
            # Apply feature selection mask
            mask_values = self.attentions[step_idx](feature)
            mask = torch.sigmoid(mask_values)
            
            # Apply mask with prior
            masked_x = mask * prior * x
            
            # Update prior for future steps
            prior = prior * (1 - mask + self.epsilon)
            prior = prior * self.gamma  # relaxation parameter
            
            # Re-encode masked input
            masked_x_encoded = self.encoder(masked_x)
            
            # Get new feature for next step
            feature = torch.split(masked_x_encoded, self.feature_dim, dim=1)[0]
            
            # Transform features for output
            step_transform = self.feature_transforms[step_idx](feature)
            step_output.append(step_transform)
        
        # Concatenate all step outputs
        output = torch.cat(step_output, dim=1)
        
        # Final prediction
        return self.final_layer(output)

# Train function for TabNet
def train_tabnet(model, X_train, y_train, X_val, y_val, target_name, batch_size=128, epochs=200, patience=20, lr=0.01):
    print(f"Training TabNet for {target_name}...")
    model.to(device)
    
    # Create data loaders
    train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    val_dataset = torch.utils.data.TensorDataset(X_val, y_val)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    
    # Loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    
    # Learning rate scheduler - use torch.optim.lr_scheduler
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=5, min_lr=1e-6, verbose=True
    )
    
    # Training loop with early stopping
    best_val_loss = float('inf')
    counter = 0
    best_model_state = None
    
    for epoch in range(epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets.unsqueeze(1))
            loss.backward()
            
            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            
            optimizer.step()
            
            train_loss += loss.item() * inputs.size(0)
        
        train_loss /= len(train_loader.dataset)
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets.unsqueeze(1))
                val_loss += loss.item() * inputs.size(0)
        
        val_loss /= len(val_loader.dataset)
        
        # Update scheduler
        scheduler.step(val_loss)
        
        # Print progress
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}")
        
        # Check for early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
            best_model_state = model.state_dict().copy()
        else:
            counter += 1
            if counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break
    
    # Load best model
    if best_model_state:
        model.load_state_dict(best_model_state)
    
    return model

# Evaluate a model
def evaluate_model(model, X_test, y_test, target_name, scaler):
    model.eval()
    X_test = X_test.to(device)
    
    with torch.no_grad():
        predictions = model(X_test).cpu().numpy().flatten()
    
    # Unscale predictions and actual values
    y_test_np = y_test.cpu().numpy()
    predictions_unscaled = scaler.inverse_transform(predictions.reshape(-1, 1)).flatten()
    y_test_unscaled = scaler.inverse_transform(y_test_np.reshape(-1, 1)).flatten()
    
    # Calculate metrics
    r2 = r2_score(y_test_unscaled, predictions_unscaled)
    rmse = math.sqrt(mean_squared_error(y_test_unscaled, predictions_unscaled))
    
    print(f"{target_name}: R² = {r2:.4f}, RMSE = {rmse:.4f}")
    return {
        'r2': r2,
        'rmse': rmse,
        'predictions': predictions_unscaled,
        'actuals': y_test_unscaled
    }

# Train all TabNet models
def train_all_tabnet_models():
    # Load and preprocess data
    (X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor, 
     X_train, X_test, y_train, y_test, feature_scaler, target_scalers) = load_data()
    
    input_dim = X_train_tensor.shape[1]
    
    # Create a TabNet model for each target
    all_results = {}
    
    for i, target_name in enumerate(TARGET_VARIABLES):
        print(f"\n{'='*20} Target: {target_name} {'='*20}")
        
        # Adjust hyperparameters based on the target
        if target_name in ['Total Cup Points', 'Total Yield']:
            # Larger models for complex targets
            feature_dim = 128
            output_dim_step = 16
            n_steps = 5
        elif target_name in ['Category One Defects', 'Quakers', 'Category Two Defects']:
            # Models for count data
            feature_dim = 96
            output_dim_step = 16
            n_steps = 4
        else:
            # Regular models for rating scales
            feature_dim = 64
            output_dim_step = 8
            n_steps = 3
        
        # Create TabNet model
        model = TabNet(
            input_dim=input_dim,
            output_dim=1,
            feature_dim=feature_dim,
            output_dim_step=output_dim_step,
            n_steps=n_steps
        )
        
        # Train model
        model = train_tabnet(
            model,
            X_train_tensor,
            y_train_tensor[:, i],
            X_test_tensor,
            y_test_tensor[:, i],
            target_name,
            batch_size=128,
            epochs=200,
            patience=20,
            lr=0.01
        )
        
        # Evaluate model
        results = evaluate_model(
            model,
            X_test_tensor,
            y_test_tensor[:, i],
            target_name,
            target_scalers[target_name]
        )
        
        all_results[target_name] = results
        all_results[target_name]['model'] = model
    
    # Print overall results
    print(f"\n{'='*20} TabNet Overall Results {'='*20}")
    r2_values = [result['r2'] for result in all_results.values()]
    
    for target_name, result in all_results.items():
        print(f"{target_name}: R² = {result['r2']:.4f}, RMSE = {result['rmse']:.4f}")
    
    avg_r2 = np.mean(r2_values)
    print(f"Average R²: {avg_r2:.4f}")
    
    return all_results

# Main function
if __name__ == "__main__":
    results = train_all_tabnet_models()

Using cuda device
Loading data...
Target value ranges:
Aroma: min=0.00, max=8.75, mean=7.63, std=0.37
Flavor: min=0.00, max=8.83, mean=7.59, std=0.39
Aftertaste: min=0.00, max=8.67, mean=7.45, std=0.39
Acidity: min=0.00, max=8.75, mean=7.59, std=0.37
Body: min=0.00, max=8.58, mean=7.55, std=0.33
Balance: min=0.00, max=8.75, mean=7.55, std=0.38
Uniformity: min=0.00, max=10.00, mean=9.89, std=0.45
Clean Cup: min=0.00, max=10.00, mean=9.86, std=0.77
Sweetness: min=0.00, max=10.00, mean=9.94, std=0.44
Cupper Points: min=0.00, max=9.00, mean=7.52, std=0.43
Total Cup Points: min=0.00, max=90.58, mean=82.60, std=3.30
Moisture: min=0.00, max=0.18, mean=0.10, std=0.04
Category One Defects: min=0.00, max=31.00, mean=0.33, std=1.73
Quakers: min=0.00, max=11.00, mean=0.13, std=0.66
Category Two Defects: min=0.00, max=47.00, mean=3.49, std=5.24
Total Yield: min=17.86, max=497666.67, mean=22925.72, std=47556.12
Input dimension: 36

==================== Target: Aroma ====================
Training Tab

In [1]:
import pickle
import joblib
import os

def save_tabnet_models(all_results, output_dir='models'):
    """
    Save all TabNet models to disk in both PyTorch (.pt) and pickle (.pkl) formats
    
    Args:
        all_results: Dictionary containing models and results
        output_dir: Directory to save the models
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    print(f"\n{'='*20} Saving Models {'='*20}")
    
    for target_name, result in all_results.items():
        model = result['model']
        
        # Clean target name for filename
        clean_name = target_name.replace(' ', '_').lower()
        
        # 1. Save using PyTorch's native format (recommended)
        torch_path = os.path.join(output_dir, f"tabnet_{clean_name}.pt")
        torch.save({
            'model_state_dict': model.state_dict(),
            'input_dim': model.input_dim,
            'output_dim': model.output_dim,
            'feature_dim': model.feature_dim,
            'output_dim_step': model.output_dim_step,
            'n_steps': model.n_steps,
            'gamma': model.gamma,
            'r2_score': result['r2'],
            'rmse': result['rmse']
        }, torch_path)
        
        # 2. Save using pickle (joblib is better for large objects)
        # Note: This works but is less reliable for PyTorch models than the native format
        pkl_path = os.path.join(output_dir, f"tabnet_{clean_name}.pkl")
        joblib.dump(model, pkl_path)
        
        print(f"Saved model for {target_name}:")
        print(f"  - PyTorch format: {torch_path}")
        print(f"  - Pickle format: {pkl_path}")
    
    # Also save the evaluation results
    results_path = os.path.join(output_dir, "tabnet_results.pkl")
    
    # Create a copy without the model objects (which can cause serialization issues)
    results_for_saving = {}
    for target_name, result in all_results.items():
        results_for_saving[target_name] = {
            'r2': result['r2'],
            'rmse': result['rmse'],
            'predictions': result['predictions'],
            'actuals': result['actuals']
        }
    
    joblib.dump(results_for_saving, results_path)
    print(f"Saved evaluation results to: {results_path}")

# Modified main function
if __name__ == "__main__":
    results = train_all_tabnet_models()
    save_tabnet_models(results)

NameError: name 'train_all_tabnet_models' is not defined